### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [3]:
# Set the base URL
nyt_url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'
# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
nyt_url_call = nyt_url + 'api-key='+ nyt_api_key + '&fq=' + filter_query + '&sort=' + sort + '&fl=' + field_list + '&begin-date' + begin_date + '&end_date' + end_date

In [4]:
# Create an empty list to store the reviews
reviews_list = []
# loop through pages 0-19
for page in range(20):
    # create query with a page number
    # API results show 10 articles at a time
    page_query = nyt_url_call + '&page=' + str(page)
    
    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(page_query).json()
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews['response']['docs']:
            reviews_list.append(review)
        # Print the page that was just retrieved            
        print(f'Checked page {page}')
    except requests.exceptions.RequestException as e:
        # Print the page number that had no results then break from the loop
        print(f'{page} had no results')
        print('Error', e)
        break

Checked page 0
Checked page 1
Checked page 2
Checked page 3
Checked page 4
Checked page 5
Checked page 6
Checked page 7
Checked page 8
Checked page 9
Checked page 10
Checked page 11
Checked page 12
Checked page 13
Checked page 14
Checked page 15
Checked page 16
Checked page 17
Checked page 18
Checked page 19


In [12]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
reviews_dump = json.dumps(reviews['response']['docs'][:5], indent = 4)
print(reviews_dump)

[
    {
        "web_url": "https://www.nytimes.com/2018/01/11/movies/lover-for-a-day-review-philippe-garrel.html",
        "snippet": "In Philippe Garrel\u2019s drama, a 23-year-old woman moves in with her father and his 23-year-old girlfriend. Complications ensue.",
        "source": "The New York Times",
        "headline": {
            "main": "Review: A Man and a Woman (and Another Woman) in \u2018Lover for a Day\u2019",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "A Love Triangle With Angles That Are Awkward",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "Lover for a Day (Movie)",
                "rank": 2,
                "major": "N"


In [13]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)
reviews_df.head(5)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2024/03/07/movies/love...,"In this neo-noir, the ever reliable, always wa...",The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2024-03-07T10:03:26+0000,986,"‘Love Lies Bleeding’ Review: Kristen Stewart, ...",Critic’s Pick,None,"Bad Romance, Hard-Boiled Style",None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
1,https://www.nytimes.com/2024/02/14/movies/blee...,Ewan McGregor plays father to his real-life da...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2024-02-14T23:21:32+0000,406,‘Bleeding Love’ Review: On the Road With Dad,None,None,"Riding Shotgun With Dad, Onscreen and in Reality",None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
2,https://www.nytimes.com/2024/02/14/movies/bob-...,This patchy biopic lauds the Marley of dormito...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2024-02-14T10:02:25+0000,926,‘Bob Marley: One Love’ Review: Mostly Positive...,None,None,"Symbol of Unity, Man of Complexity",None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None
3,https://www.nytimes.com/2024/02/08/movies/moll...,This rom-com brings futuristic absurdity and n...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2024-02-08T12:00:09+0000,275,"‘Molli and Max in the Future’ Review: Love, In...",Critic’s Pick,None,Molli and Max In the Future,None,None,None,By Nicolas Rapold,"[{'firstname': 'Nicolas', 'middlename': None, ...",None
4,https://www.nytimes.com/2024/02/08/movies/the-...,Juliette Binoche stars in an instant culinary ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2024-02-08T10:00:20+0000,1046,"‘The Taste of Things’ Review: Love, Loss and L...",Critic’s Pick,None,"At Their Table, Passion Imbues Every Meal",None,None,None,By Alissa Wilkinson,"[{'firstname': 'Alissa', 'middlename': None, '...",None


In [14]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
reviews_new_df = reviews_df
reviews_new_df['title'] = reviews_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
reviews_new_df.head()


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2024/03/07/movies/love...,"In this neo-noir, the ever reliable, always wa...",The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2024-03-07T10:03:26+0000,986,"‘Love Lies Bleeding’ Review: Kristen Stewart, ...",Critic’s Pick,None,"Bad Romance, Hard-Boiled Style",None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Love Lies Bleeding
1,https://www.nytimes.com/2024/02/14/movies/blee...,Ewan McGregor plays father to his real-life da...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2024-02-14T23:21:32+0000,406,‘Bleeding Love’ Review: On the Road With Dad,None,None,"Riding Shotgun With Dad, Onscreen and in Reality",None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Bleeding Love
2,https://www.nytimes.com/2024/02/14/movies/bob-...,This patchy biopic lauds the Marley of dormito...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2024-02-14T10:02:25+0000,926,‘Bob Marley: One Love’ Review: Mostly Positive...,None,None,"Symbol of Unity, Man of Complexity",None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None,Bob Marley: One Love
3,https://www.nytimes.com/2024/02/08/movies/moll...,This rom-com brings futuristic absurdity and n...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2024-02-08T12:00:09+0000,275,"‘Molli and Max in the Future’ Review: Love, In...",Critic’s Pick,None,Molli and Max In the Future,None,None,None,By Nicolas Rapold,"[{'firstname': 'Nicolas', 'middlename': None, ...",None,Molli and Max in the Future
4,https://www.nytimes.com/2024/02/08/movies/the-...,Juliette Binoche stars in an instant culinary ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2024-02-08T10:00:20+0000,1046,"‘The Taste of Things’ Review: Love, Loss and L...",Critic’s Pick,None,"At Their Table, Passion Imbues Every Meal",None,None,None,By Alissa Wilkinson,"[{'firstname': 'Alissa', 'middlename': None, '...",None,The Taste of Things


In [15]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_new_df['keywords'] = reviews_new_df['keywords'].apply(extract_keywords)
reviews_new_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2024/03/07/movies/love...,"In this neo-noir, the ever reliable, always wa...",The New York Times,"subject: Movies;persons: Stewart, Kristen;pers...",2024-03-07T10:03:26+0000,986,"‘Love Lies Bleeding’ Review: Kristen Stewart, ...",Critic’s Pick,None,"Bad Romance, Hard-Boiled Style",None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Love Lies Bleeding
1,https://www.nytimes.com/2024/02/14/movies/blee...,Ewan McGregor plays father to his real-life da...,The New York Times,"subject: Movies;persons: McGregor, Ewan;creati...",2024-02-14T23:21:32+0000,406,‘Bleeding Love’ Review: On the Road With Dad,None,None,"Riding Shotgun With Dad, Onscreen and in Reality",None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Bleeding Love
2,https://www.nytimes.com/2024/02/14/movies/bob-...,This patchy biopic lauds the Marley of dormito...,The New York Times,subject: Movies;subject: Reggae Music;creative...,2024-02-14T10:02:25+0000,926,‘Bob Marley: One Love’ Review: Mostly Positive...,None,None,"Symbol of Unity, Man of Complexity",None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None,Bob Marley: One Love
3,https://www.nytimes.com/2024/02/08/movies/moll...,This rom-com brings futuristic absurdity and n...,The New York Times,"subject: Movies;persons: Mamet, Zosia;creative...",2024-02-08T12:00:09+0000,275,"‘Molli and Max in the Future’ Review: Love, In...",Critic’s Pick,None,Molli and Max In the Future,None,None,None,By Nicolas Rapold,"[{'firstname': 'Nicolas', 'middlename': None, ...",None,Molli and Max in the Future
4,https://www.nytimes.com/2024/02/08/movies/the-...,Juliette Binoche stars in an instant culinary ...,The New York Times,"subject: Movies;persons: Binoche, Juliette;per...",2024-02-08T10:00:20+0000,1046,"‘The Taste of Things’ Review: Love, Loss and L...",Critic’s Pick,None,"At Their Table, Passion Imbues Every Meal",None,None,None,By Alissa Wilkinson,"[{'firstname': 'Alissa', 'middlename': None, '...",None,The Taste of Things


In [16]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
movie_title_list = reviews_new_df['title'].tolist()
movie_title_list

['Love Lies Bleeding',
 'Bleeding Love',
 'Bob Marley: One Love',
 'Molli and Max in the Future',
 'The Taste of Things',
 'Pictures of Ghosts',
 'Anyone but You',
 'Fallen Leaves',
 'Radical',
 'Fingernails',
 'My Love Affair With Marriage',
 'Foe',
 'The Creator',
 'My Sailor, My Love',
 'Cassandro',
 'Love at First Sight',
 'We Kill for Love',
 'Choose Love',
 'Before, Now & Then',
 'Love Life',
 'Red, White & Royal Blue',
 'Love in Taipei',
 'The Eternal Memory',
 'Passages',
 'Sublime',
 'Shooting Stars',
 'The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My P

### Access The Movie Database API

In [17]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [18]:
# Create an empty list to store the results

tmdb_movies_list = []
# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
for index, title in enumerate(movie_title_list):
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0:
        time.sleep(1)
        print('application is sleeping')
    # Add 1 to the request counter
    request_counter += 1
    # Perform a "GET" request for The Movie Database
    movie_request_url = url + title + tmdb_key_string
    movie_title_response = requests.get(movie_request_url).json()
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        movie_id = movie_title_response['results'][0]['id']
    
        # Make a request for a the full movie details
        movie_detail_base_url = 'https://api.themoviedb.org/3/movie/'
        full_movie_details_url = movie_detail_base_url + str(movie_id) + '?api_key=' + tmdb_api_key
        # Execute "GET" request with url
        full_movie_detail_response = requests.get(full_movie_details_url).json()
        # print(json.dumps(full_movie_detail_response, indent = 4))
        # Extract the genre names into a list
        genres_response = full_movie_detail_response['genres']
        genres = []
        for genre in genres_response:
            genres.append(genre['name'])
        # Extract the spoken_languages' English name into a list
        spoken_languages_response = full_movie_detail_response['spoken_languages']
        spoken_languages = []
        for language in spoken_languages_response:
            spoken_languages.append(language['english_name'])
        # Extract the production_countries' name into a list
        production_countries_response = full_movie_detail_response['production_countries']
        production_countries = []
        for country in production_countries_response:
            production_countries.append(country['name'])
        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        movie_details_dictionary = {'title': title,\
                                    'original title':full_movie_detail_response['original_title'],\
                                    'budget': full_movie_detail_response['budget'],\
                                    'original_language':full_movie_detail_response['original_language'],\
                                    'homepage':full_movie_detail_response['homepage'],\
                                    'overview': full_movie_detail_response['overview'],\
                                    'popularity': full_movie_detail_response['popularity'],\
                                    'runtime': full_movie_detail_response['runtime'],\
                                    'revenue': full_movie_detail_response['revenue'],\
                                    'release_date': full_movie_detail_response['release_date'],\
                                    'vote_average': full_movie_detail_response['vote_average'],\
                                    'vote_count': full_movie_detail_response['vote_count'],\
                                    'genres': genres,\
                                    'spoken_languages': spoken_languages,\
                                    'production_countries': production_countries}
        
        tmdb_movies_list.append(movie_details_dictionary)
        # Print out the title that was found
        title = movie_title_list[index]
        print(f'Found {title}')
    except:
        print(f'{title} not found')

Found Love Lies Bleeding
Found Bleeding Love
Found Bob Marley: One Love
Found Molli and Max in the Future
Found The Taste of Things
Found Pictures of Ghosts
Found Anyone but You
Found Fallen Leaves
Found Radical
Found Fingernails
Found My Love Affair With Marriage
Found Foe
Found The Creator
Found My Sailor, My Love
Found Cassandro
Found Love at First Sight
Found We Kill for Love
Found Choose Love
Found Before, Now & Then
Found Love Life
Found Red, White & Royal Blue
Found Love in Taipei
Found The Eternal Memory
Found Passages
Found Sublime
Found Shooting Stars
Found The Attachment Diaries
What’s Love Got to Do With It?’ Probably a Lo not found
Found You Can Live Forever
Found A Tourist’s Guide to Love
Found Other People’s Children
Found One True Loves
Found The Lost Weekend: A Love Story
Found A Thousand and One
Found Your Place or Mine
Found Love in the Time of Fentanyl
Found Pamela, a Love Story
Found In From the Side
Found After Love
Found Alcarràs
Found Nelly & Nadine
Found Lady C

In [19]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent = 4))

[
    {
        "title": "Love Lies Bleeding",
        "original title": "Love Lies Bleeding",
        "budget": 0,
        "original_language": "en",
        "homepage": "https://tickets.loveliesbleeding.movie",
        "overview": "Reclusive gym manager Lou falls hard for Jackie, an ambitious bodybuilder headed through town to Las Vegas in pursuit of her dream. But their love ignites violence, pulling them deep into the web of Lou\u2019s criminal family.",
        "popularity": 68.013,
        "runtime": 104,
        "revenue": 8000000,
        "release_date": "2024-03-08",
        "vote_average": 7.5,
        "vote_count": 39,
        "genres": [
            "Crime",
            "Romance",
            "Thriller"
        ],
        "spoken_languages": [
            "English"
        ],
        "production_countries": [
            "United Kingdom",
            "United States of America"
        ]
    },
    {
        "title": "Bleeding Love",
        "original title": "Love Lies Blee

In [20]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)
tmdb_df.head()

,title,original title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,Love Lies Bleeding,Love Lies Bleeding,0,en,https://tickets.loveliesbleeding.movie,Reclusive gym manager Lou falls hard for Jacki...,68.013,104,8000000,2024-03-08,7.500,39,"[Crime, Romance, Thriller]",[English],"[United Kingdom, United States of America]"
1,Bleeding Love,Love Lies Bleeding,0,en,https://tickets.loveliesbleeding.movie,Reclusive gym manager Lou falls hard for Jacki...,68.013,104,8000000,2024-03-08,7.500,39,"[Crime, Romance, Thriller]",[English],"[United Kingdom, United States of America]"
2,Bob Marley: One Love,Bob Marley: One Love,70000000,en,https://www.onelovemovie.com,Jamaican singer-songwriter Bob Marley overcome...,148.138,104,176527503,2024-02-14,6.990,497,"[Music, History, Drama]","[English, French]",[United States of America]
3,Molli and Max in the Future,Molli and Max in the Future,0,en,http://www.michaellitwak.com/molli-max-in-the-...,A sci-fi romantic comedy about a man and woman...,9.106,93,0,2024-02-09,8.667,3,"[Comedy, Romance, Science Fiction]",[English],[United States of America]
4,The Taste of Things,La Passion de Dodin Bouffant,6500000,fr,https://www.ifcfilms.com/films/the-taste-of-th...,"Set in 1889 France, Dodin Bouffant is a chef l...",128.123,135,5970753,2023-11-08,7.080,87,"[Drama, Romance]",[French],"[Belgium, France]"


### Merge and Clean the Data for Export

In [21]:
# Merge the New York Times reviews and TMDB DataFrames on title
merge_df = pd.merge(tmdb_df,reviews_new_df, on='title', )
merge_df.head()

,title,original title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,Love Lies Bleeding,Love Lies Bleeding,0,en,https://tickets.loveliesbleeding.movie,Reclusive gym manager Lou falls hard for Jacki...,68.013,104,8000000,2024-03-08,...,"‘Love Lies Bleeding’ Review: Kristen Stewart, ...",Critic’s Pick,None,"Bad Romance, Hard-Boiled Style",None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
1,Bleeding Love,Love Lies Bleeding,0,en,https://tickets.loveliesbleeding.movie,Reclusive gym manager Lou falls hard for Jacki...,68.013,104,8000000,2024-03-08,...,‘Bleeding Love’ Review: On the Road With Dad,None,None,"Riding Shotgun With Dad, Onscreen and in Reality",None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
2,Bob Marley: One Love,Bob Marley: One Love,70000000,en,https://www.onelovemovie.com,Jamaican singer-songwriter Bob Marley overcome...,148.138,104,176527503,2024-02-14,...,‘Bob Marley: One Love’ Review: Mostly Positive...,None,None,"Symbol of Unity, Man of Complexity",None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None
3,Molli and Max in the Future,Molli and Max in the Future,0,en,http://www.michaellitwak.com/molli-max-in-the-...,A sci-fi romantic comedy about a man and woman...,9.106,93,0,2024-02-09,...,"‘Molli and Max in the Future’ Review: Love, In...",Critic’s Pick,None,Molli and Max In the Future,None,None,None,By Nicolas Rapold,"[{'firstname': 'Nicolas', 'middlename': None, ...",None
4,The Taste of Things,La Passion de Dodin Bouffant,6500000,fr,https://www.ifcfilms.com/films/the-taste-of-th...,"Set in 1889 France, Dodin Bouffant is a chef l...",128.123,135,5970753,2023-11-08,...,"‘The Taste of Things’ Review: Love, Loss and L...",Critic’s Pick,None,"At Their Table, Passion Imbues Every Meal",None,None,None,By Alissa Wilkinson,"[{'firstname': 'Alissa', 'middlename': None, '...",None


In [22]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres','spoken_languages','production_countries']

# Create a list of characters to remove
characters_to_remove = ['[',']','\'']

# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    merge_df[column] = merge_df[column].astype(str)

    # Loop through characters to remove
    for character in characters_to_remove:
        merge_df[column] = merge_df[column].str.replace(character,'')
        
# Display the fixed DataFrame
merge_df.head()

,title,original title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,Love Lies Bleeding,Love Lies Bleeding,0,en,https://tickets.loveliesbleeding.movie,Reclusive gym manager Lou falls hard for Jacki...,68.013,104,8000000,2024-03-08,...,"‘Love Lies Bleeding’ Review: Kristen Stewart, ...",Critic’s Pick,None,"Bad Romance, Hard-Boiled Style",None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
1,Bleeding Love,Love Lies Bleeding,0,en,https://tickets.loveliesbleeding.movie,Reclusive gym manager Lou falls hard for Jacki...,68.013,104,8000000,2024-03-08,...,‘Bleeding Love’ Review: On the Road With Dad,None,None,"Riding Shotgun With Dad, Onscreen and in Reality",None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
2,Bob Marley: One Love,Bob Marley: One Love,70000000,en,https://www.onelovemovie.com,Jamaican singer-songwriter Bob Marley overcome...,148.138,104,176527503,2024-02-14,...,‘Bob Marley: One Love’ Review: Mostly Positive...,None,None,"Symbol of Unity, Man of Complexity",None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None
3,Molli and Max in the Future,Molli and Max in the Future,0,en,http://www.michaellitwak.com/molli-max-in-the-...,A sci-fi romantic comedy about a man and woman...,9.106,93,0,2024-02-09,...,"‘Molli and Max in the Future’ Review: Love, In...",Critic’s Pick,None,Molli and Max In the Future,None,None,None,By Nicolas Rapold,"[{'firstname': 'Nicolas', 'middlename': None, ...",None
4,The Taste of Things,La Passion de Dodin Bouffant,6500000,fr,https://www.ifcfilms.com/films/the-taste-of-th...,"Set in 1889 France, Dodin Bouffant is a chef l...",128.123,135,5970753,2023-11-08,...,"‘The Taste of Things’ Review: Love, Loss and L...",Critic’s Pick,None,"At Their Table, Passion Imbues Every Meal",None,None,None,By Alissa Wilkinson,"[{'firstname': 'Alissa', 'middlename': None, '...",None


In [23]:
# Drop "byline.person" column
merge_df = merge_df.drop(columns='byline.person')

In [24]:
# Delete duplicate rows and reset index

merge_df = merge_df.drop_duplicates()
merge_df = merge_df.reset_index(drop = True)
merge_df.head()

,title,original title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,Love Lies Bleeding,Love Lies Bleeding,0,en,https://tickets.loveliesbleeding.movie,Reclusive gym manager Lou falls hard for Jacki...,68.013,104,8000000,2024-03-08,...,986,"‘Love Lies Bleeding’ Review: Kristen Stewart, ...",Critic’s Pick,None,"Bad Romance, Hard-Boiled Style",None,None,None,By Manohla Dargis,None
1,Bleeding Love,Love Lies Bleeding,0,en,https://tickets.loveliesbleeding.movie,Reclusive gym manager Lou falls hard for Jacki...,68.013,104,8000000,2024-03-08,...,406,‘Bleeding Love’ Review: On the Road With Dad,None,None,"Riding Shotgun With Dad, Onscreen and in Reality",None,None,None,By Manohla Dargis,None
2,Bob Marley: One Love,Bob Marley: One Love,70000000,en,https://www.onelovemovie.com,Jamaican singer-songwriter Bob Marley overcome...,148.138,104,176527503,2024-02-14,...,926,‘Bob Marley: One Love’ Review: Mostly Positive...,None,None,"Symbol of Unity, Man of Complexity",None,None,None,By Amy Nicholson,None
3,Molli and Max in the Future,Molli and Max in the Future,0,en,http://www.michaellitwak.com/molli-max-in-the-...,A sci-fi romantic comedy about a man and woman...,9.106,93,0,2024-02-09,...,275,"‘Molli and Max in the Future’ Review: Love, In...",Critic’s Pick,None,Molli and Max In the Future,None,None,None,By Nicolas Rapold,None
4,The Taste of Things,La Passion de Dodin Bouffant,6500000,fr,https://www.ifcfilms.com/films/the-taste-of-th...,"Set in 1889 France, Dodin Bouffant is a chef l...",128.123,135,5970753,2023-11-08,...,1046,"‘The Taste of Things’ Review: Love, Loss and L...",Critic’s Pick,None,"At Their Table, Passion Imbues Every Meal",None,None,None,By Alissa Wilkinson,None


In [26]:
# Export data to CSV without the index
merge_df.to_csv('movie_data.csv', index=False)